### Import libraries and features 

In [10]:
import gc
import sys
import os 
import pandas as pd 
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [5]:
home = os.path.expanduser("~")
sys.path.append(f"{home}/Documents/projects/CarPriceRegression/Machine_Learning/scripts/")

In [6]:
from util import *
%load_ext autoreload
%autoreload 2

### Import data and split into train and test 

In [12]:
data = pd.read_csv(f"{home}/Documents/projects/CarPriceRegression/Processed_Data/car_onehot_data.csv")
features, y = data.drop("price",axis=1), data.price
X_train, y_train, X_test, y_test = train_test_split(features,y,test_size=0.2,random_state=2020)

In [13]:
X_train.shape,y_train.shape, X_test.shape,y_test.shape

((79348, 206), (19838, 206), (79348,), (19838,))

### garbage collect

In [14]:
gc.collect()
%reset -f out

Flushing output cache (2 entries)
